<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/ANATOMY_AGENTIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""DB-BOB-MVP-AGENT-DEMO-PROTO.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1hfpcZJWMBzZ1S6CS-UKfHqXer1E08wD_

## libraries
"""

## libraries  --- Installation ---

!pip install flask -q
!pip install flask_migrate -q
!pip install flask_sqlalchemy -q
!pip install anthropic -q
!pip install colab-env --quiet
!pip install flask_cors -q



!pip install flask_ngrok -q
!pip install pyngrok -q  # Install pyngrok directly
!pip install --upgrade flask_ngrok pyngrok -q
!pip install flask_login -q

!pip install Flask-Login -q


!pip install flask_login -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# Install Flask and pyngrok
!pip install Flask pyngrok -q
!pip install colab-env -q

In [2]:
from flask import Flask, render_template_string, request, jsonify
from pyngrok import ngrok
import os
import threading
import time # Import time for a small delay
import colab_env # Import colab_env for environment variables

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
from flask import Flask, render_template_string, request, jsonify
from pyngrok import ngrok
import os
import time
import socket # To find a free port if needed


# --- Google Colab / Gemini API Imports and Configuration ---
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    import google.generativeai as genai
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google Generative AI configured successfully using Colab Secrets.")
except ImportError:
    print("Not running in Google Colab. Attempting to get 'GEMINI' environment variable.")
    GOOGLE_API_KEY = os.getenv('GEMINI')
    if not GOOGLE_API_KEY:
        raise ValueError("GEMINI API Key not found. Please set it as an environment variable or Colab Secret.")
    import google.generativeai as genai
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google Generative AI configured successfully using environment variable.")

# --- Configuration for Agent ---
class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-1.5-flash" # Set the desired Gemini model

# Define the HTML content as a Python string.
HTML_CONTENT = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Agentic AI Anatomy Demo</title>
    <!-- Tailwind CSS CDN -->
    <script src="https://cdn.tailwindcss.com"></script>
    <!-- Inter Font -->
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap" rel="stylesheet">
    <!-- React, ReactDOM, and Babel CDNs for in-browser JSX compilation -->
    <script src="https://unpkg.com/react@18/umd/react.development.js"></script>
    <script src="https://unpkg.com/react-dom@18/umd/react-dom.development.js"></script>
    <script src="https://unpkg.com/@babel/standalone/babel.min.js"></script>
    <style>
        body {
            font-family: 'Inter', sans-serif;
        }
    </style>
</head>
<body>
    <div id="root"></div>

    <script type="text/babel">
        // React App Component
        const App = () => {
            const [status, setStatus] = React.useState('Idle');
            const [currentPerception, setCurrentPerception] = React.useState('');
            const [memoryContent, setMemoryContent] = React.useState([]);
            const [reasoningOutput, setReasoningOutput] = React.useState('');
            const [actionTaken, setActionTaken] = React.useState('');
            const [learningUpdate, setLearningUpdate] = React.useState('');
            const [loopRunning, setLoopRunning] = React.useState(false);
            const intervalRef = React.useRef(null);

            const perceptions = [
                "Summarize a flight plan for a trip from Montreal to Paris, focusing on key milestones and considerations.",
                "Generate a short story about an AI agent that learns to appreciate art.",
                "Explain the concept of quantum entanglement in simple terms.",
                "Describe the ideal environment for growing tomatoes.",
                "Provide a recipe for a healthy vegan dinner."
            ];

            React.useEffect(() => {
                if (loopRunning) {
                    intervalRef.current = setInterval(() => {
                        runAgentLoop();
                    }, 5000);
                } else {
                    clearInterval(intervalRef.current);
                }
                return () => clearInterval(intervalRef.current);
            }, [loopRunning]);

            const runAgentLoop = async () => {
                setStatus('Perceiving...');
                const newPerception = perceptions[Math.floor(Math.random() * perceptions.length)];
                setCurrentPerception(newPerception);

                setTimeout(() => {
                    setStatus('Recalling Memory...');
                    setMemoryContent(prev => {
                        const updatedMemory = [...prev, `[${new Date().toLocaleTimeString()}] Perceived: ${newPerception}`];
                        return updatedMemory.slice(-5);
                    });

                    setTimeout(async () => {
                        setStatus('Reasoning & Planning (via Gemini 1.5 Flash API)...');
                        setReasoningOutput('Contacting AI...');
                        setActionTaken('Waiting for AI response...');

                        try {
                            const response = await fetch('/api/reason', {
                                method: 'POST',
                                headers: {
                                    'Content-Type': 'application/json',
                                },
                                body: JSON.stringify({ prompt: newPerception }),
                            });

                            const result = await response.json();

                            if (response.ok) {
                                const aiResponse = result.response;
                                const displayResponse = typeof aiResponse === 'string' ? aiResponse.substring(0, 100) : JSON.stringify(aiResponse).substring(0, 100);
                                const displayAction = typeof aiResponse === 'string' ? aiResponse.substring(0, 50) : JSON.stringify(aiResponse).substring(0, 50);

                                setReasoningOutput(`AI analyzed: "${newPerception}". AI Response: ${displayResponse}...`);
                                setActionTaken(`AI generated a detailed response. Ready to act based on this: "${displayAction}..."`);
                            } else {
                                setReasoningOutput(`AI reasoning failed: ${result.error}`);
                                setActionTaken(`Failed to get AI action.`);
                            }
                        } catch (error) {
                            setReasoningOutput(`Error during AI reasoning: ${error.message}`);
                            setActionTaken(`Action failed due to error.`);
                        }

                        setTimeout(() => {
                            setStatus('Executing Action...');
                            setTimeout(() => {
                                setStatus('Learning & Adapting...');
                                const learningFeedback = `Learned from processing "${newPerception}" and integrating AI reasoning.`;
                                setLearningUpdate(learningFeedback);
                                setTimeout(() => {
                                    setStatus('Idle (Ready for next cycle)');
                                }, 1000);
                            }, 1000);
                        }, 1000);
                    }, 1000);
                }, 1000);
            };

            const toggleLoop = () => {
                setLoopRunning(prev => !prev);
                if (loopRunning) {
                    setCurrentPerception('');
                    setMemoryContent([]);
                    setReasoningOutput('');
                    setActionTaken('');
                    setLearningUpdate('');
                    setStatus('Idle');
                }
            };

            return (
                <div className="min-h-screen bg-gradient-to-br from-indigo-500 via-purple-600 to-pink-500 text-white p-6 font-inter flex flex-col items-center">
                    <h1 className="text-4xl font-bold mb-8 text-center drop-shadow-lg">
                        Agentic AI Anatomy Demo
                    </h1>
                    <h2 className="text-2xl font-semibold mb-8 text-center text-purple-200">
                        A Gemini 1.5 Flash Perspective (with API Integration)
                    </h2>

                    {/* Control Buttons */}
                    <div className="flex space-x-4 mb-8">
                        <button
                            onClick={toggleLoop}
                            className={`px-8 py-3 rounded-full font-bold text-lg transition-all duration-300 ease-in-out transform hover:scale-105 shadow-lg
                                ${loopRunning ? 'bg-red-600 hover:bg-red-700' : 'bg-green-500 hover:bg-green-600'}`}
                        >
                            {loopRunning ? 'Stop Agent' : 'Start Agent'}
                        </button>
                        <button
                            onClick={() => runAgentLoop()}
                            disabled={loopRunning}
                            className="px-8 py-3 rounded-full font-bold text-lg bg-blue-500 hover:bg-blue-600 transition-all duration-300 ease-in-out transform hover:scale-105 shadow-lg disabled:opacity-50 disabled:cursor-not-allowed"
                        >
                            Run One Cycle
                        </button>
                    </div>

                    {/* Agent Status */}
                    <div className="bg-gradient-to-r from-purple-800 to-indigo-800 p-4 rounded-xl shadow-2xl mb-8 w-full max-w-2xl text-center">
                        <p className="text-xl font-semibold">Agent Status: <span className="text-yellow-300">{status}</span></p>
                    </div>

                    {/* Anatomy Components */}
                    <div className="grid grid-cols-1 md:grid-cols-2 gap-6 w-full max-w-4xl">
                        {/* Perception */}
                        <div className="bg-white bg-opacity-10 p-5 rounded-xl shadow-xl border border-purple-400 flex flex-col">
                            <h3 className="text-xl font-bold mb-3 text-purple-200">1. Perception & Sensing</h3>
                            <div className="bg-gray-800 bg-opacity-70 text-green-300 p-4 rounded-lg flex-grow flex items-center justify-center text-center">
                                <p className="font-mono text-lg">{currentPerception || 'Awaiting input...'}</p>
                            </div>
                        </div>

                        {/* Memory */}
                        <div className="bg-white bg-opacity-10 p-5 rounded-xl shadow-xl border border-purple-400 flex flex-col">
                            <h3 className="text-xl font-bold mb-3 text-purple-200">2. Memory & Knowledge</h3>
                            <div className="bg-gray-800 bg-opacity-70 text-blue-300 p-4 rounded-lg overflow-y-auto h-32">
                                {memoryContent.length > 0 ? (
                                    memoryContent.map((item, index) => (
                                        <p key={index} className="font-mono text-sm mb-1">{item}</p>
                                    ))
                                ) : (
                                    <p className="font-mono text-sm text-center italic">Memory is clear...</p>
                                )}
                            </div>
                        </div>

                        {/* Reasoning */}
                        <div className="bg-white bg-opacity-10 p-5 rounded-xl shadow-xl border border-purple-400 flex flex-col">
                            <h3 className="text-xl font-bold mb-3 text-purple-200">3. Reasoning & Decision-Making</h3>
                            <div className="bg-gray-800 bg-opacity-70 text-orange-300 p-4 rounded-lg flex-grow flex items-center justify-center text-center">
                                <p className="font-mono text-lg">{reasoningOutput || 'Analyzing input...'}</p>
                            </div>
                        </div>

                        {/* Action */}
                        <div className="bg-white bg-opacity-10 p-5 rounded-xl shadow-xl border border-purple-400 flex flex-col">
                            <h3 className="text-xl font-bold mb-3 text-purple-200">4. Action & Execution</h3>
                            <div className="bg-gray-800 bg-opacity-70 text-red-300 p-4 rounded-lg flex-grow flex items-center justify-center text-center">
                                <p className="font-mono text-lg">{actionTaken || 'Awaiting decision...'}</p>
                            </div>
                        </div>

                        {/* Learning */}
                        <div className="col-span-1 md:col-span-2 bg-white bg-opacity-10 p-5 rounded-xl shadow-xl border border-purple-400 flex flex-col">
                            <h3 className="text-xl font-bold mb-3 text-purple-200">5. Learning & Adaptation</h3>
                            <div className="bg-gray-800 bg-opacity-70 text-cyan-300 p-4 rounded-lg flex-grow flex items-center justify-center text-center">
                                <p className="font-mono text-lg">{learningUpdate || 'Adapting and improving...'}</p>
                            </div>
                        </div>
                    </div>

                    <p className="mt-8 text-sm text-gray-300 text-center">
                        This demo simulates a simplified agentic loop: perceive, store in memory, reason, act, and learn.
                        The 'Reasoning' step now integrates with the Gemini API via the Flask backend for more dynamic responses.
                    </p>
                </div>
            );
        };

        // Mount the React App
        const container = document.getElementById('root');
        const root = ReactDOM.createRoot(container);
        root.render(<App />);
    </script>
</body>
</html>
"""

app = Flask(__name__)

@app.route('/')
def index():
    """Serves the main HTML content."""
    return render_template_string(HTML_CONTENT)

@app.route('/api/reason', methods=['POST'])
def reason():
    """
    Handles reasoning requests by sending the prompt to the Gemini 1.5 Flash model
    and returning its generated response.
    """
    if not GOOGLE_API_KEY:
        return jsonify({"error": "Gemini API key not configured on the backend."}), 500

    data = request.get_json()
    prompt = data.get('prompt', 'Generate a general AI response.')

    try:
        model = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
        response = model.generate_content(prompt)
        generated_text = ""
        if response.parts:
            generated_text = response.text
        elif response.candidates and len(response.candidates) > 0 and response.candidates[0].content:
             generated_text = response.candidates[0].content.parts[0].text
        else:
            generated_text = "No text content found in Gemini response."
        return jsonify({"response": generated_text})
    except Exception as e:
        print(f"Error calling Gemini API: {e}")
        if "safety" in str(e).lower() or "blocked" in str(e).lower():
            return jsonify({"error": f"AI response blocked due to safety concerns or content policy. Please try a different prompt. Error: {e}"}), 400
        else:
            return jsonify({"error": f"Failed to get response from AI: {e}"}), 500

# Function to find a free port
def find_free_port():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('0.0.0.0', 0)) # Bind to port 0 to let OS find a free port
    port = sock.getsockname()[1]
    sock.close()
    return port


Google Generative AI configured successfully using Colab Secrets.


In [14]:
#--- Application Startup ---
if __name__ == '__main__':
    # Ensure any previous ngrok tunnels are killed before starting a new one
    ngrok.kill()

    try:
        # Get authtoken from environment variable
        authtoken = None
        try:
            from google.colab import userdata
            authtoken = userdata.get('NGROK_TOKEN')
        except (ImportError, KeyError):
            authtoken = os.getenv('NGROK_TOKEN')

        if not authtoken:
            raise ValueError("NGROK_TOKEN environment variable not set. Please set it in Colab's Secrets (key icon) with 'Notebook access' ON, or as an environment variable.")

        ngrok.set_auth_token(authtoken)
        print("ngrok authtoken loaded.")

        # Find a free port for Flask to use
        flask_port = find_free_port()
        print(f"Flask will attempt to run on dynamically assigned port: {flask_port}")

        # Establish the ngrok tunnel to the dynamically assigned Flask port
        print(f"Attempting to establish ngrok tunnel to port {flask_port}...")
        public_url = ngrok.connect(flask_port).public_url
        print(f"ngrok tunnel started at: {public_url}")
        print("Click the link above to view the demo!")

        # Run Flask app in the main thread (blocking call)
        # This will now use the same port ngrok is tunneling to
        print(f"Starting Flask app on port {flask_port}...")
        app.run(host="0.0.0.0", port=flask_port, use_reloader=False, debug=True)

    except Exception as e:
        print(f"An error occurred during setup or execution: {e}")
        print("Please ensure your ngrok authtoken (NGROK_TOKEN) and Gemini API key (GEMINI) are correctly set in Colab Secrets, and that port is available.")
    finally:
        ngrok.kill()
        print("ngrok tunnel disconnected.")

ngrok authtoken loaded.
Flask will attempt to run on dynamically assigned port: 48355
Attempting to establish ngrok tunnel to port 48355...
ngrok tunnel started at: https://942c-34-42-113-87.ngrok-free.app
Click the link above to view the demo!
Starting Flask app on port 48355...
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:48355
 * Running on http://172.28.0.12:48355
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an

ngrok tunnel disconnected.
